# Set up

In [1]:
!git clone https://github.com/NLP-Reichman/assignment_1.git
!mv assignment_1/data data
!rm assignment_1/ -r

fatal: destination path 'assignment_1' already exists and is not an empty directory.
'mv' is not recognized as an internal or external command,
operable program or batch file.


'rm' is not recognized as an internal or external command,
operable program or batch file.


# Introduction
In this assignment you will be creating tools for learning and testing language models. The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.
The relevant files are under the data folder:

- en.csv (or the equivalent JSON file)
- es.csv (or the equivalent JSON file)
- fr.csv (or the equivalent JSON file)
- in.csv (or the equivalent JSON file)
- it.csv (or the equivalent JSON file)
- nl.csv (or the equivalent JSON file)
- pt.csv (or the equivalent JSON file)
- tl.csv (or the equivalent JSON file)

In [2]:
import json
# from google.colab import files
import pandas as pd
import numpy as np
import glob
import sys
from collections import Counter


# Implementation

## Part 1
Implement the function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. Our token definition is a single UTF-8 encoded character. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

Note - do NOT lowercase the sentences in whi HW.

In [3]:
def preprocess() -> list[str]:
  '''
  Return a list of characters, representing the shared vocabulary of all languages
  '''
  df = pd.concat(map(pd.read_csv, glob.glob('assignment_1/data/*.csv')))
  unique_chars = list(''.join(set(df['tweet_text'].str.cat(sep=''))))
  # unique_chars.append('<br>')
  # unique_chars.append('</br>')
  return unique_chars

unique_chars = preprocess()
V = len(unique_chars)
V


1803

## Part 2
Implement the function *lm* that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant *n*-1 sequences, and the values are dictionaries with the *n*_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{ "ab":{"c":0.5, "b":0.25, "d":0.25}, "ca":{"a":0.2, "b":0.7, "d":0.1} }

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [4]:
def lm(lang: str, n: int) -> dict[str, dict[str, float]]:
    '''
    Return a language model for the given lang and n_gram (n)
    :param lang: the language of the model
    :param n: the n_gram value
    :return: a dictionary where the keys are n_grams and the values are dictionaries
    '''
    df = pd.read_csv('assignment_1/data/' + lang + '.csv')
    # print(df.head())
    text = ''.join(df['tweet_text'].str.cat(sep=''))

    lm = {}

    for i in range(len(text)-n):
        ngram = text[i:i+n-1]
        suffix = text[i+n-1]

        if ngram not in lm:
            lm[ngram] = {}
            lm[ngram][suffix] = 1

        else:
            if suffix in lm[ngram]:
                lm[ngram][suffix] += 1
            else:
                lm[ngram][suffix] = 1

    # print(lm)

    for ngram in lm:
        sum_ngram = sum(lm[ngram].values())
        for suffix in lm[ngram]:
            lm[ngram][suffix] = lm[ngram][suffix] / sum_ngram

    # print(lm)

    return lm


ngrams = lm('en', 3)


## Part 3
Implement the function *eval* that returns the perplexity of a model (dictionary) running over the data file of the given target language.

In [5]:
def eval(model: dict, target_lang: str) -> float:
  '''
  Return the perplexity value calculated over applying the model on the text file
  of the target_lang language.
  :param model: the language model
  :param target_lang: the target language
  :return: the perplexity value
  '''
  df = pd.read_csv('assignment_1/data/' + target_lang + '.csv')
  # for row in df.iterrows():
  #   print(row)
  sum_perplexity = 0
  for index, row in df.iterrows():
    # print(row['tweet_text'])
    sum_perplexity += perplexity(row['tweet_text'], model)
  
  sum_perplexity /= (len(df) - 1)
  return sum_perplexity

def perplexity(sentence, model):
  n = len(next(iter((model))))
  length = len(sentence)
  perp = 1

  for i in range(len(sentence)-n):
    ngram = sentence[i:i+n]
    suffix = sentence[i+n]
    # print(ngram, suffix)

    if ngram in model:
      if suffix in model[ngram]:
        perp *= model[ngram][suffix]
      else: perp *= 1/V
    else: perp *= 1/V
  # print(sentence)
  if perp == 0: perp = sys.float_info.min
  perp = 1/perp
  perp = perp**(1/length)
  return perp


## Part 4
Implement the *match* function that calls *eval* using a specific value of *n* for every possible language pair among the languages we have data for. You should call *eval* for every language pair four times, with each call assign a different value for *n* (1-4). Each language pair is composed of the source language and the target language. Before you make the call, you need to call the *lm* function to create the language model for the source language. Then you can call *eval* with the language model and the target language. The function should return a pandas DataFrame with the following four columns: *source_lang*, *target_lang*, *n*, *perplexity*. The values for the first two columns are the two-letter language codes. The value for *n* is the *n* you use for generating the specific perplexity values which you should store in the forth column.

In [6]:
def match() -> pd.DataFrame:
  '''
  Return a DataFrame containing one line per every language pair and n_gram.
  Each line will contain the perplexity calculated when applying the language model
  of the source language on the text of the target language.
  :return: a DataFrame containing the perplexity values
  '''

  languages = ['en', 'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']
  n_grams = [1, 2, 3, 4]
  # languages = languages[:2]
  # n_grams = n_grams[:2]

  data = []
  for source_lang in languages:
    for n in n_grams:
      model = lm(source_lang, n)
      for target_lang in languages:
        print(source_lang, n, target_lang)
        data.append([source_lang, target_lang, n, eval(model, target_lang)])

  return pd.DataFrame(data, columns=['source_lang', 'target_lang', 'n_gram', 'perplexity'])

match1 = match()

en 1 en
en 1 es
en 1 fr
en 1 in
en 1 it
en 1 nl
en 1 pt
en 1 tl
en 2 en
en 2 es
en 2 fr
en 2 in
en 2 it
en 2 nl
en 2 pt
en 2 tl
en 3 en
en 3 es
en 3 fr
en 3 in
en 3 it
en 3 nl
en 3 pt
en 3 tl
en 4 en
en 4 es
en 4 fr
en 4 in
en 4 it
en 4 nl
en 4 pt
en 4 tl
es 1 en
es 1 es
es 1 fr
es 1 in
es 1 it
es 1 nl
es 1 pt
es 1 tl
es 2 en
es 2 es
es 2 fr
es 2 in
es 2 it
es 2 nl
es 2 pt
es 2 tl
es 3 en
es 3 es
es 3 fr
es 3 in
es 3 it
es 3 nl
es 3 pt
es 3 tl
es 4 en
es 4 es
es 4 fr
es 4 in
es 4 it
es 4 nl
es 4 pt
es 4 tl
fr 1 en
fr 1 es
fr 1 fr
fr 1 in
fr 1 it
fr 1 nl
fr 1 pt
fr 1 tl
fr 2 en
fr 2 es
fr 2 fr
fr 2 in
fr 2 it
fr 2 nl
fr 2 pt
fr 2 tl
fr 3 en
fr 3 es
fr 3 fr
fr 3 in
fr 3 it
fr 3 nl
fr 3 pt
fr 3 tl
fr 4 en
fr 4 es
fr 4 fr
fr 4 in
fr 4 it
fr 4 nl
fr 4 pt
fr 4 tl
in 1 en
in 1 es
in 1 fr
in 1 in
in 1 it
in 1 nl
in 1 pt
in 1 tl
in 2 en
in 2 es
in 2 fr
in 2 in
in 2 it
in 2 nl
in 2 pt
in 2 tl
in 3 en
in 3 es
in 3 fr
in 3 in
in 3 it
in 3 nl
in 3 pt
in 3 tl
in 4 en
in 4 es
in 4 fr
in 4 in
in 4 it


In [7]:
# match1.to_csv('match1.csv', index=False)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(match1.loc[match1['n_gram'] == 3].sort_values(by='perplexity', ascending=False))


    source_lang target_lang  n_gram  perplexity
144          it          en       3         inf
85           fr          nl       3         inf
112          in          en       3         inf
116          in          it       3         inf
118          in          pt       3         inf
119          in          tl       3         inf
149          it          nl       3         inf
150          it          pt       3         inf
151          it          tl       3         inf
176          nl          en       3         inf
179          nl          in       3         inf
180          nl          it       3         inf
182          nl          pt       3         inf
183          nl          tl       3         inf
208          pt          en       3         inf
211          pt          in       3         inf
213          pt          nl       3         inf
243          tl          in       3         inf
215          pt          tl       3         inf
87           fr          tl       3     

## Part 5
Implement the *generate* function which takes a language code, *n*, the prompt (the starting text), the number of tokens to generate, and *r*, which is the random seed for any randomized action you plan to take in your implementation. The function should start generating tokens, one by one, using the language model of the given source language and *n*. The prompt should be used as a starting point for aligning on the probabilities to be used for generating the next token.

Note - The generation of the next token should be from the LM's distribution.

In [8]:
def generate(lang: str, n: int, prompt: str, number_of_tokens: int, r: int) -> str:
  '''
  Generate text in the given language using the given parameters.
  :param lang: the language of the model
  :param n: the n_gram value
  :param prompt: the prompt to start the generation
  :param number_of_tokens: the number of tokens to generate
  :param r: the random seed to use
  '''
  model = lm(lang, n)
  length = len(prompt)
  print(prompt)
  for i in range(number_of_tokens):
    # print(i)
    ngram = prompt[-n+1:]
    # print(ngram)
    prompt += random_from_distribution_dict(model[ngram])
    print(prompt)

  return prompt

def random_from_distribution_dict(dct):
    rand_val = np.random.rand()
    total = 0
    for k, v in dct.items():
        total += v
        if rand_val <= total:
            return k
    assert False, 'unreachable'



prompt = 'how are you friend?'
len(generate('en', 5, prompt, 200, 1))

how are you friend?
how are you friend? 
how are you friend? h
how are you friend? ht
how are you friend? htt
how are you friend? http
how are you friend? https
how are you friend? https:
how are you friend? https:/
how are you friend? https://
how are you friend? https://t
how are you friend? https://t.
how are you friend? https://t.c
how are you friend? https://t.co
how are you friend? https://t.co/
how are you friend? https://t.co/p
how are you friend? https://t.co/pn
how are you friend? https://t.co/pnQ
how are you friend? https://t.co/pnQ1
how are you friend? https://t.co/pnQ1S
how are you friend? https://t.co/pnQ1Sn
how are you friend? https://t.co/pnQ1Sn8
how are you friend? https://t.co/pnQ1Sn88
how are you friend? https://t.co/pnQ1Sn880
how are you friend? https://t.co/pnQ1Sn880V
how are you friend? https://t.co/pnQ1Sn880VR
how are you friend? https://t.co/pnQ1Sn880VRT
how are you friend? https://t.co/pnQ1Sn880VRT 
how are you friend? https://t.co/pnQ1Sn880VRT @
how are you fr

219

## Part 6
Play with your generate function, try to generate different texts in different language and various values of *n*. No need to submit anything of that.

# Testing

Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [9]:
####################
# PLACE TESTS HERE #

# Create tests
def test_preprocess():
    return {
        'vocab_length': len(preprocess()),
    }

def test_lm():
    return {
        'english_2_gram_length': len(lm('en', 2)),
        'english_3_gram_length': len(lm('en', 3)),
        'french_3_gram_length': len(lm('fr', 3)),
        'spanish_3_gram_length': len(lm('es', 3)),
    }

def test_eval():
    return {
        'english_on_english': round(eval(lm('en', 3), 'en'), 2),
        'english_on_french': round(eval(lm('en', 3), 'fr'), 2),
        'english_on_spanish': round(eval(lm('en', 3), 'es'), 2),
    }

def test_match():
    df = match()
    return {
        'df_shape': df.shape,
        'en_en_1': df[(df['source'] == 'en') & (df['target'] == 'en') & (df['n'] == 1)]['perplexity'].values[0],
        'tl_tl_1': df[(df['source'] == 'tl') & (df['target'] == 'tl') & (df['n'] == 1)]['perplexity'].values[0],
        'tl_nl_4': df[(df['source'] == 'tl') & (df['target'] == 'nl') & (df['n'] == 4)]['perplexity'].values[0],
    }

def test_generate():
    return {
        'english_2_gram': generate('en', 2, "I am", 20, 5),
        'english_3_gram': generate('en', 3, "I am", 20, 5),
        'english_4_gram': generate('en', 4, "I Love", 20, 5),
        'spanish_2_gram': generate('es', 2, "Soy", 20, 5),
        'spanish_3_gram': generate('es', 3, "Soy", 20, 5),
        'french_2_gram': generate('fr', 2, "Je suis", 20, 5),
        'french_3_gram': generate('fr', 3, "Je suis", 20, 5),
    }

TESTS = [test_preprocess, test_lm, test_eval, test_match, test_generate]

# Run tests and save results
res = {}
for test in TESTS:
    print(f'Running test: {test.__name__}')
    try:
        cur_res = test()
        res.update({test.__name__: cur_res})
    except Exception as e:
        res.update({test.__name__: repr(e)})


# save the result file as results.json
with open('results.json', 'w') as f:
    json.dump(res, f, indent=2)


# with open('results.json', 'w') as f:
#     json.dump(res, f, indent=2)

# Download the results.json file
# files.download('results.json')

####################

Running test: test_preprocess
Running test: test_lm
Running test: test_eval
Running test: test_match
en 1 en
en 1 es
en 1 fr
en 1 in
en 1 it
en 1 nl
en 1 pt
en 1 tl
en 2 en
en 2 es
en 2 fr
en 2 in
en 2 it
en 2 nl
en 2 pt
en 2 tl
en 3 en
en 3 es
en 3 fr
en 3 in
en 3 it
en 3 nl
en 3 pt
en 3 tl
en 4 en
en 4 es
en 4 fr
en 4 in
en 4 it
en 4 nl
en 4 pt
en 4 tl
es 1 en
es 1 es
es 1 fr
es 1 in
es 1 it
es 1 nl
es 1 pt
es 1 tl
es 2 en
es 2 es
es 2 fr
es 2 in
es 2 it
es 2 nl
es 2 pt
es 2 tl
es 3 en
es 3 es
es 3 fr
es 3 in
es 3 it
es 3 nl
es 3 pt
es 3 tl
es 4 en
es 4 es
es 4 fr
es 4 in
es 4 it
es 4 nl
es 4 pt
es 4 tl
fr 1 en
fr 1 es
fr 1 fr
fr 1 in
fr 1 it
fr 1 nl
fr 1 pt
fr 1 tl
fr 2 en
fr 2 es
fr 2 fr
fr 2 in
fr 2 it
fr 2 nl
fr 2 pt
fr 2 tl
fr 3 en
fr 3 es
fr 3 fr
fr 3 in
fr 3 it
fr 3 nl
fr 3 pt
fr 3 tl
fr 4 en
fr 4 es
fr 4 fr
fr 4 in
fr 4 it
fr 4 nl
fr 4 pt
fr 4 tl
in 1 en
in 1 es
in 1 fr
in 1 in
in 1 it
in 1 nl
in 1 pt
in 1 tl
in 2 en
in 2 es
in 2 fr
in 2 in
in 2 it
in 2 nl
in 2 pt
in 2 tl
in 

In [10]:
# Show the local files, results.json should be there now and
# also downloaded to your local machine
!ls -l

'ls' is not recognized as an internal or external command,
operable program or batch file.
